In [1]:
from importlib import reload
import honours_project.simulation as simulation
reload(simulation)
import honours_project.data as data
reload(data)

import json

In [149]:
from honours_project.data import PNE, Cycle, SimulationResult

# Example of problematic seed
sim = simulation.Simulation(5, 2, seed=1737044418)
result1 = sim.run()
result2 = sim.run()
print(json.dumps(result1, indent=4, cls=data.EnhancedJSONEncoder))
print(json.dumps(result2, indent=4, cls=data.EnhancedJSONEncoder))

{
    "time": 0.00491637499362696,
    "iteration": 2
}
{
    "time": 0.00156958399747964,
    "iteration": 0
}
